# Squat quality scoring with TensorFlow

This notebook trains a small BlazePose + LSTM regressor on your local squat videos and predicts a quality score per clip.


## 1) Environment setup
- Installs TensorFlow + MediaPipe + OpenCV.
- Use your own data/squats_train and data/squats_test folders; no downloads required.


In [ ]:
!python -m pip install --upgrade pip
!python -m pip install "tensorflow<2.17" tensorflow-io jupyter
!python -m pip install mediapipe opencv-python


## 2) Imports and configuration
- Adjust paths or scoring scale if needed.
- Scores are expected in data/squat_scores.csv.


In [ ]:
import os
import random
import csv
from pathlib import Path

import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# Seed for deterministic behavior where possible (shuffling, TF ops)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Training / preprocessing hyper-parameters and constants
BATCH_SIZE = 16
NUM_FRAMES = 16            # number of frames sampled per video (temporal length)
IMG_SIZE = 160             # image-ops size used in earlier versions (not used by pose extractor)
NUM_LANDMARKS = 33         # BlazePose outputs 33 landmarks per frame
LANDMARK_DIMS = 4          # per-landmark dims: (x, y, z, visibility)
SCORE_SCALE = 100.0        # label scale for human readable scores (labels stored 0..100, model trains on 0..1)

# Paths used throughout the notebook
DATA_ROOT = Path("data")
TRAIN_DIR = DATA_ROOT / "squats_train"
TEST_DIR = DATA_ROOT / "squats_test"
LABELS_PATH = DATA_ROOT / "squat_scores.csv"
MODEL_DIR = Path("checkpoints")

# Allowed video extensions when scanning directories
VIDEO_EXTS = (".mp4", ".mov", ".avi", ".mkv")

# Ensure disk layout exists before proceeding
MODEL_DIR.mkdir(parents=True, exist_ok=True)
TRAIN_DIR.mkdir(parents=True, exist_ok=True)
TEST_DIR.mkdir(parents=True, exist_ok=True)

# Helpful prints for users running the notebook interactively
print('TensorFlow version:', tf.__version__)
print('Data root:', DATA_ROOT.resolve())
print('Model dir:', MODEL_DIR.resolve())

2.16.2
Data root: C:\Users\KarthikPC\vscode_projects\CS663_Project2_training\data
Model dir: C:\Users\KarthikPC\vscode_projects\CS663_Project2_training\checkpoints


## 3) Prepare local data and labels
- A CSV template is generated listing every video under data/squats_train and data/squats_test.
- Fill in the score column (0-100). At least two labeled train videos are required to run.


In [2]:
def list_videos(root: Path):
    return sorted(
        p for p in root.rglob("*")
        if p.suffix.lower() in VIDEO_EXTS and p.is_file()
    )


def ensure_label_file():
    existing = {}
    if LABELS_PATH.exists():
        with LABELS_PATH.open("r", newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing[row.get("relative_path", "")] = row.get("score", "")

    rows = []
    for p in list_videos(TRAIN_DIR) + list_videos(TEST_DIR):
        rel = p.relative_to(DATA_ROOT).as_posix()
        rows.append({"relative_path": rel, "score": existing.get(rel, "")})

    LABELS_PATH.parent.mkdir(parents=True, exist_ok=True)
    with LABELS_PATH.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["relative_path", "score"])
        writer.writeheader()
        writer.writerows(rows)

    print(f"Label file ready at {LABELS_PATH}. Fill in 'score' (0-{int(SCORE_SCALE)}) for each row.")
    return rows


_ = ensure_label_file()


Label file ready at data\squat_scores.csv. Fill in 'score' (0-100) for each row.


### Label summary / sanity check


In [3]:
def load_labeled_samples():
    samples = []
    missing = []
    with LABELS_PATH.open("r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            rel = row.get("relative_path", "")
            score_str = row.get("score", "").strip()
            if not rel:
                continue

            full = DATA_ROOT / rel
            if not full.exists():
                missing.append(rel)
                continue

            if not score_str:
                continue

            try:
                score = float(score_str)
            except ValueError:
                print(f"Skipping {rel}: invalid score '{score_str}'")
                continue

            score = max(0.0, min(SCORE_SCALE, score))
            samples.append((str(full), score))

    if missing:
        print("Warning: paths not found on disk:", missing)

    print(f"Loaded {len(samples)} labeled samples.")
    return samples


labeled_samples = load_labeled_samples()
if len(labeled_samples) < 2:
    raise ValueError("Add scores in the CSV (at least 2 labeled videos) before training.")


Loaded 132 labeled samples.


## 4) Build a TensorFlow input pipeline
- Uniformly sample frames, run BlazePose to get 33 landmarks per frame, normalize, and emit flattened keypoints.
- Labels are normalized to 0-1 during training; final scores are rescaled to 0-100.


In [ ]:
mp_pose = mp.solutions.pose


def _sample_frame_indices(total_frames: int, num_target: int) -> np.ndarray:
    """Return `num_target` frame indices sampled evenly across the video.

    Notes:
      - If total_frames is 0 => returns zeros (preserves dtype/shape expected by callers).
      - Returned dtype is np.int32 to match indexing expectations in downstream code.
    """
    if total_frames <= 0:
        return np.zeros((num_target,), dtype=np.int32)
    idxs = np.linspace(0, max(total_frames - 1, 0), num_target).astype(np.int32)
    return idxs


def _normalize_landmarks(landmarks: np.ndarray) -> np.ndarray:
    """Robust per-frame normalization & rotation for landmarks.

    Input:
      - landmarks: shape (NUM_LANDMARKS, LANDMARK_DIMS) with columns (x, y, z, visibility)
    Output:
      - normalized landmarks (same shape) where:
          * coordinates are centered (hip midpoint or visible mean)
          * scaled by torso/hip size to produce scale-invariant features
          * rotated so the major body axis is horizontal (helps the model focus on pose shape)
      - returns None if there are fewer than two sufficiently-visible points in the frame

    The function uses visibility thresholds and layered fallbacks (hip midpoint -> shoulders -> PCA)
    so the pipeline remains robust with occasional missing detections.
    """
    # Use visibility flag (4th column) to decide which points are reliable
    vis = landmarks[:, 3] >= 0.25
    if vis.sum() < 2:
        # Not enough points — prefer to skip this frame (caller can interpolate later)
        return None

    def valid_pair(i, j):
        try:
            return bool(vis[i] and vis[j])
        except Exception:
            return False

    LEFT_HIP, RIGHT_HIP = 23, 24
    LEFT_SHOULDER, RIGHT_SHOULDER = 11, 12

    # Choose a stable center point: hip midpoint if available, else mean of visible coords
    if valid_pair(LEFT_HIP, RIGHT_HIP):
        left_hip, right_hip = landmarks[LEFT_HIP, :3], landmarks[RIGHT_HIP, :3]
        center_hip = (left_hip + right_hip) / 2.0
    else:
        visible_coords = landmarks[vis, :3]
        center_hip = visible_coords.mean(axis=0)

    # Choose a shoulder center if possible (used to compute scale/torso length)
    if valid_pair(LEFT_SHOULDER, RIGHT_SHOULDER):
        left_sh, right_sh = landmarks[LEFT_SHOULDER, :3], landmarks[RIGHT_SHOULDER, :3]
        center_sh = (left_sh + right_sh) / 2.0
    elif valid_pair(LEFT_HIP, RIGHT_HIP):
        # crude fallback: estimate shoulder position above the hip center
        center_sh = center_hip + np.array([0.0, 0.5, 0.0])
    else:
        center_sh = center_hip + np.array([0.0, 0.5, 0.0])

    # scale estimate — torso length or hip separation (pixels normalized to frame size by BlazePose)
    torso = np.linalg.norm(center_sh[:2] - center_hip[:2])
    hip_dist = np.linalg.norm(landmarks[LEFT_HIP, :2] - landmarks[RIGHT_HIP, :2]) if valid_pair(LEFT_HIP, RIGHT_HIP) else torso
    scale = max(torso, hip_dist, 1e-3)

    # shift & scale (in-place) -> avoid mutating caller by copying earlier in pipeline
    landmarks[:, :3] = (landmarks[:, :3] - center_hip) / scale

    # compute rotation vector: prefer hips -> shoulders -> PCA fallback
    if valid_pair(LEFT_HIP, RIGHT_HIP):
        hip_vec = (landmarks[RIGHT_HIP, :2] - landmarks[LEFT_HIP, :2])
    elif valid_pair(LEFT_SHOULDER, RIGHT_SHOULDER):
        hip_vec = (landmarks[RIGHT_SHOULDER, :2] - landmarks[LEFT_SHOULDER, :2])
    else:
        visible_pts = landmarks[vis, :2]
        if visible_pts.shape[0] < 2:
            hip_vec = np.array([1.0, 0.0])
        else:
            pts_centered = visible_pts - visible_pts.mean(axis=0)
            u, s, vh = np.linalg.svd(pts_centered, full_matrices=False)
            hip_vec = vh[0]

    # rotate so the main body axis is horizontal (small epsilon avoids division-by-zero)
    angle = np.arctan2(hip_vec[1], hip_vec[0] + 1e-6)
    cos_a, sin_a = np.cos(-angle), np.sin(-angle)
    rot = np.array([[cos_a, -sin_a], [sin_a, cos_a]], dtype=np.float32)
    landmarks[:, :2] = landmarks[:, :2] @ rot.T
    return landmarks


def _extract_keypoints_np(video_path: str) -> np.ndarray:
    """Extract normalized per-frame keypoint features for NUM_FRAMES timesteps.

    Output is a (NUM_FRAMES, NUM_LANDMARKS * LANDMARK_DIMS + 1) array where the
    last column is a per-frame validity flag (1.0 = original frame had pose, 0.0 = interpolated).

    The function implements caching (sha1 of the path) to speed repeated runs on the
    same video. Missing frames are temporally interpolated so the sequence is dense.
    """
    import hashlib

    cache_dir = MODEL_DIR / "keypoints_cache"
    cache_dir.mkdir(parents=True, exist_ok=True)
    h = hashlib.sha1(video_path.encode("utf-8")).hexdigest()
    cache_file = cache_dir / f"{h}.npy"
    if cache_file.exists():
        try:
            return np.load(str(cache_file))
        except Exception:
            pass

    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    num_frames = len(frames)
    keypoints = np.full((NUM_FRAMES, NUM_LANDMARKS, LANDMARK_DIMS), np.nan, dtype=np.float32)
    if num_frames == 0:
        # nothing to do — return zeros + invalid mask
        filled = np.zeros((NUM_FRAMES, NUM_LANDMARKS * LANDMARK_DIMS), dtype=np.float32)
        valid_mask = np.zeros((NUM_FRAMES,), dtype=np.float32)
        out = np.concatenate([filled, valid_mask[:, None]], axis=1)
        np.save(str(cache_file), out)
        return out

    idxs = _sample_frame_indices(num_frames, NUM_FRAMES)

    mp_pose_local = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        smooth_landmarks=True,
    )

    valid_per_frame = np.zeros((NUM_FRAMES,), dtype=bool)
    for out_i, frame_idx in enumerate(idxs):
        frame = frames[int(frame_idx)]
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mp_pose_local.process(image_rgb)
        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            coords = np.array([[p.x, p.y, p.z, p.visibility] for p in lm], dtype=np.float32)
            norm = _normalize_landmarks(coords)
            if norm is not None:
                keypoints[out_i] = norm
                valid_per_frame[out_i] = True

    try:
        mp_pose_local.close()
    except Exception:
        pass

    # interpolate missing values for each landmark/dimension across the temporal axis
    idxs_time = np.arange(NUM_FRAMES)
    for li in range(NUM_LANDMARKS):
        for d in range(LANDMARK_DIMS):
            series = keypoints[:, li, d]
            good = ~np.isnan(series)
            if good.any():
                keypoints[:, li, d] = np.interp(idxs_time, idxs_time[good], series[good])
            else:
                keypoints[:, li, d] = 0.0

    frame_valid = valid_per_frame.astype(np.float32)

    # Flatten and append the per-frame validity flag as an extra feature column
    keypoints_flat = keypoints.reshape((NUM_FRAMES, NUM_LANDMARKS * LANDMARK_DIMS))
    out = np.concatenate([keypoints_flat, frame_valid[:, None]], axis=1).astype(np.float32)

    try:
        np.save(str(cache_file), out)
    except Exception:
        pass

    return out


def load_keypoints(path: tf.Tensor) -> tf.Tensor:
    """Wrapper for tf.data pipeline.

    - Accepts a tf.Tensor path and uses tf.py_function to call the Python preprocessing above.
    - Returns a tf.Tensor with shape (NUM_FRAMES, NUM_LANDMARKS * LANDMARK_DIMS + 1) and dtype tf.float32.
    """
    def _py_decode(p):
        return _extract_keypoints_np(p.numpy().decode("utf-8"))

    kpts = tf.py_function(_py_decode, [path], tf.float32)
    # shape = NUM_FRAMES x (NUM_LANDMARKS * LANDMARK_DIMS + 1)
    kpts.set_shape((NUM_FRAMES, NUM_LANDMARKS * LANDMARK_DIMS + 1))
    return kpts


def preprocess(path: tf.Tensor, score: tf.Tensor) -> tuple[tf.Tensor, tf.Tensor]:
    """Dataset mapping function.

    - Loads the keypoints for `path` (video path), returns (features, label) pair.
    - The label is normalized to 0..1 (model training range) and reshaped to (1,) per sample.
    """
    keypoints = load_keypoints(path)
    # already (NUM_FRAMES, features); no reshape needed
    score = tf.cast(score, tf.float32) / SCORE_SCALE
    score = tf.expand_dims(score, axis=-1)
    return keypoints, score


def build_tf_dataset(samples, training: bool):
    """Build a batched tf.data.Dataset from (path, score) samples.

    Notes:
      - When training=True the dataset is shuffled deterministically (seeded by SEED)
      - Uses AUTOTUNE for parallel preprocessing
    """
    paths, scores = zip(*samples)
    ds = tf.data.Dataset.from_tensor_slices((list(paths), list(scores)))
    if training:
        ds = ds.shuffle(buffer_size=len(paths), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

Train batches: 5
Val batches: 2
Test batches: 2


## 5) Define a lightweight regression model
- BiLSTM + pooling over landmark sequences; single sigmoid output predicts normalized score.


In [5]:
FEATURE_DIMS = NUM_LANDMARKS * LANDMARK_DIMS + 1

def build_model() -> tf.keras.Model:
    inputs = tf.keras.Input(shape=(NUM_FRAMES, FEATURE_DIMS))
    # no global masking — we append a per-frame "valid" flag as one feature
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(inputs)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)  # normalized score 0-1
    return tf.keras.Model(inputs, outputs)


model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="mse",
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")],
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 133)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 16, 256)        │       268,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 309,505 (1.18 MB)

 Trainable params: 309,505 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

## 6) Train
- Early stopping on validation MAE.


In [6]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor="val_mae"),
    tf.keras.callbacks.ModelCheckpoint(str(MODEL_DIR / "model.keras"), save_best_only=True, monitor="val_mae"),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=400,
    callbacks=callbacks,
)

best_val_mae = min(history.history["val_mae"])
print("Best val MAE (normalized 0-1):", best_val_mae)
print("Best val MAE (score units):", best_val_mae * SCORE_SCALE)


Epoch 1/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: 0.1843 - mae: 0.4100 - val_loss: 0.1469 - val_mae: 0.3580
Epoch 2/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: 0.1843 - mae: 0.4100 - val_loss: 0.1469 - val_mae: 0.3580
Epoch 2/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1426 - mae: 0.3501 - val_loss: 0.1274 - val_mae: 0.3198
Epoch 3/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1426 - mae: 0.3501 - val_loss: 0.1274 - val_mae: 0.3198
Epoch 3/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1162 - mae: 0.3027 - val_loss: 0.1194 - val_mae: 0.2895
Epoch 4/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1162 - mae: 0.3027 - val_loss: 0.1194 - val_mae: 0.2895
Epoch 4/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1104 - mae: 0.2796 - val_loss: 0.1146 - val_mae: 0.2713
Epoch 5/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1104 - mae: 0.2796 - val_loss: 0.1146 - val_mae: 0.2713
Epoch 5/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1013 - mae: 0.

## 7) Evaluate and save artifacts


In [7]:
eval_target = test_ds if test_ds is not None else val_ds
eval_results = model.evaluate(eval_target, return_dict=True)
print(eval_results)
print(f"MAE in score units: {eval_results['mae'] * SCORE_SCALE:.2f}")

# Save both a single-file Keras archive and a SavedModel directory for robust conversion.
keras_export_file = MODEL_DIR / "squat_scorer.keras"
saved_model_dir = MODEL_DIR / "squat_scorer_savedmodel"

print('Saving single-file Keras archive ->', keras_export_file)
model.save(str(keras_export_file))

# Keras 3: prefer model.export(...) to write a SavedModel dir; fall back to tf.saved_model.save
try:
    print('Exporting SavedModel (model.export) ->', saved_model_dir)
    model.export(str(saved_model_dir))
except Exception:
    try:
        print('Fallback export via tf.saved_model.save ->', saved_model_dir)
        tf.saved_model.save(model, str(saved_model_dir))
    except Exception as e:
        print('SavedModel export failed:', e)

with (MODEL_DIR / "score_scale.txt").open("w", encoding="utf-8") as f:
    f.write(str(SCORE_SCALE))

print("Artifacts saved to", MODEL_DIR)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 748ms/step - loss: 0.0457 - mae: 0.1627
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 748ms/step - loss: 0.0457 - mae: 0.1627
{'loss': 0.045712172985076904, 'mae': 0.16267424821853638}
MAE in score units: 16.27
Saving single-file Keras archive -> checkpoints\squat_scorer.keras
Exporting SavedModel (model.export) -> checkpoints\squat_scorer_savedmodel
{'loss': 0.045712172985076904, 'mae': 0.16267424821853638}
MAE in score units: 16.27
Saving single-file Keras archive -> checkpoints\squat_scorer.keras
Exporting SavedModel (model.export) -> checkpoints\squat_scorer_savedmodel
INFO:tensorflow:Assets written to: checkpoints\squat_scorer_savedmodel\assets
INFO:tensorflow:Assets written to: checkpoints\squat_scorer_savedmodel\assets


INFO:tensorflow:Assets written to: checkpoints\squat_scorer_savedmodel\assets


Saved artifact at 'checkpoints\squat_scorer_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 16, 133), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1430656486384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656487088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656485680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656486208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656486912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656486736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656487792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656485856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656487264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656489024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1430656488496: Tens

## 8) Export TFLite for Android


In [ ]:
from pathlib import Path

MODEL_DIR = Path("checkpoints")
saved_model_dir = MODEL_DIR / "squat_scorer_savedmodel"  # or wherever you exported
tflite_path = MODEL_DIR / "squat_scorer.tflite"

# Create converter from SavedModel -> TFLite. SavedModel is preferred for complex models
# because it preserves the computation graph and provides a robust conversion path.
converter = tf.lite.TFLiteConverter.from_saved_model(str(saved_model_dir))
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Key flags for TensorList + LSTM
# - experimental_enable_resource_variables: helps the converter handle resource variables
# - _experimental_lower_tensor_list_ops: controls lowering of TensorList ops
# - target_spec.supported_ops: adding SELECT_TF_OPS allows Flex ops to be included for
#   TF-only ops (e.g., while/tensorlist constructs used by LSTM). Note that SELECT_TF_OPS
#   requires the TensorFlow flex runtime on the target (increases APK/binary size).
converter.experimental_enable_resource_variables = True
converter._experimental_lower_tensor_list_ops = False
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]

# Convert and write.
tflite_model = converter.convert()
tflite_path.write_bytes(tflite_model)
print("Wrote", tflite_path)

Wrote checkpoints\squat_scorer.tflite


## 9) Single-sample inference helper


In [ ]:
def predict_sample(video_path: str):
    """Single-sample prediction helper for interactive testing.

    - Accepts: path to a video file on disk
    - Returns: predicted score in human units (0..SCORE_SCALE)

    The function uses the same preprocessing pipeline to extract the NUM_FRAMES keypoint
    sequence and reshapes it to the model input shape: (1, NUM_FRAMES, FEATURE_DIMS).
    """
    keypoints = _extract_keypoints_np(video_path)
    # keypoints shape: (NUM_FRAMES, FEATURE_DIMS) where FEATURE_DIMS = NUM_LANDMARKS*LANDMARK_DIMS + 1
    keypoints = keypoints.reshape(1, NUM_FRAMES, NUM_LANDMARKS * LANDMARK_DIMS + 1)

    # model.predict returns a normalized scalar in 0..1; scale up to the human-readable range
    score_norm = float(model.predict(keypoints, verbose=0)[0][0])
    return score_norm * SCORE_SCALE


# Example usage (interactive): pick a path from train_samples if available
example_path = train_samples[0][0] if train_samples else labeled_samples[0][0]
pred_score = predict_sample(example_path)
print(f"Predicted score: {pred_score:.2f} (0-{int(SCORE_SCALE)}) on {example_path}")